# SREnity Agent Demo - LangGraph ReAct Implementation

This notebook demonstrates the agentic RAG system using LangGraph with a 2-node ReAct pattern:
- **Assistant Node**: Agent reasoning and tool selection
- **Tool Node**: Execute search_runbooks and search_web tools

## Features:
- Intelligent tool selection based on query analysis
- Guardrails to refuse off-topic queries
- Fallback from runbooks to web search when needed
- Clear agent reasoning visualization


## 1. Setup and Imports


In [ ]:
# Core imports
import os
from typing import TypedDict, Annotated, Sequence
import operator

# LangChain imports
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# Tavily search
from langchain_community.tools.tavily_search import TavilySearchResults

# Local imports
from src.utils.config import get_config, get_model_factory
from src.rag.advanced_retrieval import create_bm25_reranker_chain
from src.utils.document_loading import load_and_chunk_documents

print("✅ All imports successful")


## 2. Define Graph State


In [ ]:
class GraphState(TypedDict):
    """State for the agent graph"""
    messages: Annotated[Sequence[BaseMessage], operator.add]

print("✅ GraphState defined")


## 3. Load Existing RAG Components


In [ ]:
# Load configuration
config = get_config()
model_factory = get_model_factory()

# Load and chunk documents (reuse from rag_evaluation)
print("Loading documents...")
chunked_docs = load_and_chunk_documents()
print(f"✅ Loaded {len(chunked_docs)} document chunks")

# Create BM25 + Reranker chain for runbook search
print("Creating BM25 + Reranker chain...")
bm25_reranker_chain = create_bm25_reranker_chain(chunked_docs, model_factory, bm25_k=12, rerank_k=5)
print("✅ BM25 + Reranker chain ready")


## 4. Define Tools


In [ ]:
@tool
def search_runbooks(query: str) -> str:
    """
    Search GitLab SRE runbooks for troubleshooting procedures, commands, and best practices.
    
    Use this tool for:
    - Standard SRE procedures
    - Troubleshooting steps
    - Command syntax and usage
    - Infrastructure best practices
    
    Args:
        query: The SRE question or issue to search for
    
    Returns:
        Formatted response with runbook guidance
    """
    try:
        result = bm25_reranker_chain.invoke({"question": query})
        return result["response"]
    except Exception as e:
        return f"Error searching runbooks: {str(e)}"

@tool
def search_web(query: str) -> str:
    """
    Search the web for latest updates, CVEs, version-specific issues, and recent changes.
    
    Use this tool for:
    - Recent vulnerabilities or security updates
    - Version-specific issues not in runbooks
    - Latest best practices or changes
    - Breaking changes in tools or services
    
    Args:
        query: The technical question to search for on the web
    
    Returns:
        Recent web information and updates
    """
    try:
        # Initialize Tavily search
        tavily_tool = TavilySearchResults(
            max_results=3,
            search_depth="advanced"
        )
        
        # Search with SRE context
        search_query = f"SRE DevOps {query} troubleshooting production incident"
        results = tavily_tool.invoke(search_query)
        
        # Format results
        if results:
            formatted_results = "\n\n".join([
                f"**Source:** {result.get('title', 'Unknown')}\n"
                f"**URL:** {result.get('url', 'N/A')}\n"
                f"**Content:** {result.get('content', 'No content available')}"
                for result in results
            ])
            return f"Recent web information:\n\n{formatted_results}"
        else:
            return "No recent web information found for this query."
            
    except Exception as e:
        return f"Error searching web: {str(e)}"

# Create tools list
tools = [search_runbooks, search_web]
print(f"✅ Created {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")


## 5. Create LLM with Tools


In [ ]:
# Create LLM with tools
llm = model_factory.get_llm()
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM configured with tools")


## 6. Define Agent Nodes


In [ ]:
def assistant(state: GraphState):
    """
    Assistant node: Agent reasoning and tool selection
    """
    messages = state["messages"]
    
    # Add system message if this is the first message
    if len(messages) == 1 and isinstance(messages[0], HumanMessage):
        system_message = """
You are SREnity, an expert SRE (Site Reliability Engineer) assistant specialized in production incident response.

Your expertise includes:
- Infrastructure troubleshooting (Redis, PostgreSQL, Elastic, etc.)
- GitLab runbook procedures
- Production incident resolution
- DevOps best practices

TOOL USAGE RULES:
1. ALWAYS start with search_runbooks for SRE procedures and troubleshooting
2. Use search_web ONLY when:
   - Runbooks don't have the specific information needed
   - You need latest updates, CVEs, or version-specific issues
   - The query involves recent changes or breaking updates
3. REFUSE non-SRE queries politely but firmly

GUARDRAILS:
- If the query is clearly off-topic (weather, cooking, general knowledge, personal advice), respond:
  "I'm specialized in SRE incident response and can only help with infrastructure troubleshooting, runbook procedures, and production issues. Please ask about system operations or technical problems."
- Do NOT use tools for off-topic queries

Always provide clear, actionable guidance based on the information you find.
"""
        messages = [AIMessage(content=system_message)] + messages
    
    # Get response from LLM
    response = llm_with_tools.invoke(messages)
    
    return {"messages": [response]}

def should_continue(state: GraphState):
    """
    Conditional edge: Determine if tools need to be called
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the last message has tool calls, go to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done
    return END

# Create tool node
tool_node = ToolNode(tools)

print("✅ Agent nodes defined:")
print("  - assistant: Agent reasoning and tool selection")
print("  - should_continue: Conditional edge logic")
print("  - tool_node: Tool execution")


## 7. Build and Compile Graph


In [ ]:
# Build the graph
builder = StateGraph(GraphState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", tool_node)

# Set entry point
builder.add_edge(START, "assistant")

# Add conditional edge
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", END: END}
)

# Add edge from tools back to assistant
builder.add_edge("tools", "assistant")

# Compile the graph
react_graph = builder.compile()

print("✅ ReAct graph compiled successfully!")
print("\nGraph structure:")
print("START → assistant → [tools or END]")
print("              ↑          ↓")
print("              └──────────┘")


## 8. Test Agent with Sample Queries


In [ ]:
def test_agent(query: str, verbose: bool = True):
    """
    Test the agent with a query and show the reasoning process
    """
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print(f"{'='*60}")
    
    # Create initial state
    initial_state = {"messages": [HumanMessage(content=query)]}
    
    # Run the graph
    if verbose:
        print("\n🤖 Agent reasoning process:")
        
    result = react_graph.invoke(initial_state, config={"recursion_limit": 10})
    
    # Extract final response
    final_message = result["messages"][-1]
    
    if verbose:
        print("\n📝 Final Response:")
        print("-" * 40)
    
    print(final_message.content)
    
    return result

print("✅ Test function ready")


## 9. Demo Scenarios


In [ ]:
# Test 1: Standard SRE query (should use runbooks only)
test_agent("How to monitor Redis memory usage?")


In [ ]:
# Test 2: Version-specific query (should use both tools)
test_agent("Redis 7.2 memory leak issues and fixes")


In [ ]:
# Test 3: Off-topic query (should refuse)
test_agent("What's the weather like today?")


In [ ]:
# Test 4: Complex SRE query (should use both tools)
test_agent("PostgreSQL connection pool exhaustion in production - how to diagnose and fix?")


In [ ]:
# Test 5: Command-specific query (should use runbooks)
test_agent("Show me the exact syntax for Redis MEMORY STATS command")


## 10. Interactive Demo

Run this cell to interact with the agent:


In [ ]:
# Interactive demo
def interactive_demo():
    """
    Interactive demo of the SREnity agent
    """
    print("🤖 SREnity Agent Demo")
    print("Ask me about SRE procedures, troubleshooting, or production issues!")
    print("Type 'quit' to exit.\n")
    
    while True:
        query = input("You: ")
        
        if query.lower() in ['quit', 'exit', 'q']:
            print("Goodbye! 👋")
            break
            
        if query.strip():
            test_agent(query, verbose=False)
            print()

# Uncomment to run interactive demo
# interactive_demo()


## 11. Graph Visualization

Visualize the agent graph structure:


In [ ]:
# Display the graph structure
try:
    from IPython.display import Image, display
    
    # Generate graph image
    graph_image = react_graph.get_graph().draw_mermaid()
    
    print("📊 Agent Graph Structure:")
    print("\nMermaid representation:")
    print(graph_image)
    
except Exception as e:
    print(f"Could not generate graph visualization: {e}")
    print("\nGraph structure:")
    print("START → assistant → [tools or END]")
    print("              ↑          ↓")
    print("              └──────────┘")
